<a href="https://colab.research.google.com/github/gianmarcomejia96/UTEC_DATA_DISCOVERY/blob/main/fuentes_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install requests beautifulsoup4
#pip install scrapy
#pip install selenium
#pip install playwright
#pip install sqlalchemy psycopg2
#pip install pymongo

# Lectura y adquisición de datos

Este notebook explora diferentes métodos para **leer y adquirir** desde diversas fuentes:
- Bases de datos relacionales (PostgreSQL) y NoSQL (MongoDB)
- Web scraping con BeautifulSoup, Scrapy, Selenium y Playwright
- APIs REST

In [ ]:
import pandas as pd
import numpy as np

### 1) Bases de Datos

#### - Ejemplo con PostgreSQL

En este ejemplo nos conectamos a una base de datos **PostgreSQL** local:
1. Establecemos la conexión usando credenciales
2. Creamos una tabla llamada `people` con columnas id, name y age
3. Insertamos datos de ejemplo
4. Consultamos los datos y los convertimos a un DataFrame de pandas

In [ ]:
#psql -U postgres
#CREATE DATABASE "mydatabase";

import psycopg2

# Paso 1: Conectarse a la base de datos PostgreSQL
connection = psycopg2.connect(
    dbname="postgres",  # Nombre de tu base de datos
    user="postgres",      # Nombre de usuario en PostgreSQL
    password="admin",      # Contraseña de tu usuario
    host="localhost",     # Host (en este caso es local)
    port="5432"           # Puerto por defecto de PostgreSQL
)

# Crear un cursor para interactuar con la base de datos
cursor = connection.cursor()

# Paso 2: Crear una tabla llamada "people" con columnas "name" y "age"
create_table_query = """
CREATE TABLE IF NOT EXISTS people (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    age INT NOT NULL
);
"""
cursor.execute(create_table_query)  # Ejecutar la consulta para crear la tabla
connection.commit()  # Confirmar los cambios

# Paso 3: Insertar datos en la tabla "people"
insert_data_query = """
INSERT INTO people (name, age)
VALUES
    ('Alice', 30),
    ('Bob', 25),
    ('Charlie', 35);
"""
cursor.execute(insert_data_query)  # Ejecutar la consulta para insertar datos
connection.commit()  # Confirmar los cambios

# Paso 4: Seleccionar todos los datos de la tabla "people"
select_query = "SELECT * FROM people;"
cursor.execute(select_query)  # Ejecutar la consulta SELECT

# Paso 5: Obtener todos los resultados
rows = cursor.fetchall()  # Obtener todos los resultados de la consulta

# Paso 6: Convertir los resultados en un DataFrame de Pandas
# Crear un DataFrame de Pandas usando las filas obtenidas de la consulta
df = pd.DataFrame(rows, columns=["ID", "Name", "Age"])

# Paso 7: Cerrar el cursor y la conexión a la base de datos
cursor.close()  # Cerrar el cursor
connection.close()  # Cerrar la conexión
df.head(3)

OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


#### - Ejemplo con MongoDB

**MongoDB** es una base de datos NoSQL orientada a documentos. En este ejemplo:
1. Nos conectamos a MongoDB local
2. Creamos/accedemos a una base de datos llamada `moviesdb`
3. Importamos datos desde un archivo JSON que contiene información de películas

In [ ]:
import json
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')

db = client['moviesdb']
collection = db['movies']

with open('movies.json', 'r', encoding='utf-8') as file:
    # Cargar los datos del archivo JSON
    movies_data = json.load(file)

# Insertar los datos en la colección
if isinstance(movies_data, list):  # Si el JSON es un array de documentos
    collection.insert_many(movies_data)
else:  # Si el JSON contiene un único documento
    collection.insert_one(movies_data)

print("Documentos importados exitosamente!")

ModuleNotFoundError: No module named 'pymongo'

Una vez importados los datos, podemos realizar consultas usando la sintaxis de MongoDB.

In [ ]:
# Contar el número de documentos en la colección
num_documents = collection.count_documents({})
print(f"Número total de documentos: {num_documents}")

# Contar películas antes del año 1901
count_before_1901 = collection.count_documents({"year": {"$lt": 1901}})
print(f"\nNúmero de películas lanzadas antes de 1901: {count_before_1901}")

Número total de documentos: 36273

Número de películas lanzadas antes de 1901: 18


Contamos el total de documentos y filtramos películas antiguas (antes de 1901) usando operadores de consulta de MongoDB (`$lt` = less than).

In [ ]:
# Mostrar películas antes del año 1901
consulta = collection.find({"year": {"$lt": 1901}}).limit(5)
for documento in consulta:
    print(documento)

{'_id': ObjectId('6966e1458c68c7a89f11e03a'), 'title': 'After Dark in Central Park', 'year': 1900, 'cast': [], 'genres': [], 'href': None}
{'_id': ObjectId('6966e1458c68c7a89f11e03b'), 'title': "Boarding School Girls' Pajama Parade", 'year': 1900, 'cast': [], 'genres': [], 'href': None}
{'_id': ObjectId('6966e1458c68c7a89f11e03c'), 'title': "Buffalo Bill's Wild West Parad", 'year': 1900, 'cast': [], 'genres': [], 'href': None}
{'_id': ObjectId('6966e1458c68c7a89f11e03d'), 'title': 'Caught', 'year': 1900, 'cast': [], 'genres': [], 'href': None}
{'_id': ObjectId('6966e1458c68c7a89f11e03e'), 'title': 'Clowns Spinning Hats', 'year': 1900, 'cast': [], 'genres': ['Silent'], 'href': 'Clowns_Spinning_Hats', 'extract': 'Clowns Spinning Hats is a black-and-white silent film featuring clowns throwing hats back and forth to each other. It was written and produced by Lubin Films and released April 7, 1900.'}


Mostramos las primeras 5 películas que cumplen con el filtro para inspeccionar su estructura.

In [ ]:
# Ejecutamos la consulta para obtener todos los documentos de la colección
consulta = collection.find()  # Esto devuelve todos los documentos de la colección
# Convertimos el cursor a una lista de diccionarios
data = list(consulta)
# Creamos un DataFrame a partir de los datos obtenidos
df = pd.DataFrame(data)
df.head()

,_id,title,year,cast,genres,href,extract,thumbnail,thumbnail_width,thumbnail_height
0,6966e1458c68c7a89f11e03a,After Dark in Central Park,1900,[],[],None,NaN,NaN,NaN,NaN
1,6966e1458c68c7a89f11e03b,Boarding School Girls' Pajama Parade,1900,[],[],None,NaN,NaN,NaN,NaN
2,6966e1458c68c7a89f11e03c,Buffalo Bill's Wild West Parad,1900,[],[],None,NaN,NaN,NaN,NaN
3,6966e1458c68c7a89f11e03d,Caught,1900,[],[],None,NaN,NaN,NaN,NaN
4,6966e1458c68c7a89f11e03e,Clowns Spinning Hats,1900,[],[Silent],Clowns_Spinning_Hats,Clowns Spinning Hats is a black-and-white sile...,NaN,NaN,NaN


Extraemos todos los documentos de la colección y los convertimos a un DataFrame de pandas para análisis posterior.

In [ ]:
# Asegurémonos de que las columnas 'thumbnail_width', 'thumbnail_height', 'title' y 'year' no tengan valores nulos
df_filtered = df.dropna(subset=['title', 'year', 'thumbnail_width', 'thumbnail_height'])
# Filtramos solo las columnas 'title', 'year', 'thumbnail_width', 'thumbnail_height'
df_filtered = df_filtered[['title', 'year', 'thumbnail_width', 'thumbnail_height']]
# Ordenamos el DataFrame por 'thumbnail_width' de forma descendente
df_sorted = df_filtered.sort_values(by="year", ascending=False)
df_sorted_reset = df_sorted.reset_index(drop=True)
df_sorted_reset

,title,year,thumbnail_width,thumbnail_height
0,Beau Is Afraid,2023,250.0,370.0
1,Aquaman and the Lost Kingdom,2023,320.0,163.0
2,Trolls Band Together,2023,251.0,397.0
3,Fast X,2023,220.0,348.0
4,Monica,2023,250.0,357.0
...,...,...,...,...
30363,Love by the Light of the Moon,1901,320.0,227.0
30364,The Enchanted Drawing,1900,320.0,240.0
30365,Capture of Boer Battery by British,1900,320.0,240.0
30366,"Searching Ruins on Broadway, Galveston, for De...",1900,320.0,240.0


### 2) Webscrapping

#### - Usando BeautifulSoup

**BeautifulSoup** es una librería para extraer datos de HTML/XML. Es ideal para páginas web estáticas.

En este ejemplo:
1. Hacemos una solicitud HTTP a una página web
2. Parseamos el HTML con BeautifulSoup
3. Extraemos información específica (citas y autores) usando selectores CSS
4. Convertimos los datos a un DataFrame

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL objetivo
url = "https://quotes.toscrape.com/"

# Hacer la solicitud HTTP a la página
response = requests.get(url)
if response.status_code == 200:
    print("Conexión exitosa!")
else:
    print(f"Error al conectarse: {response.status_code}")

# Parsear el contenido HTML de la página
soup = BeautifulSoup(response.content, "html.parser")

# Extraer datos: en este caso, citas y autores
quotes = soup.find_all("div", class_="quote")

data = []
for quote in quotes:
    text = quote.find("span", class_="text").get_text(strip=True)
    author = quote.find("small", class_="author").get_text(strip=True)
    data.append({"quote": text, "author": author})

# Convertir los datos a un DataFrame de Pandas
df = pd.DataFrame(data)

# Mostrar las primeras filas
print(df.head())

# Guardar los datos en un archivo CSV
#df.to_csv("quotes.csv", index=False)
#print("Datos guardados en quotes.csv")

Conexión exitosa!
                                               quote           author
0  “The world as we have created it is a process ...  Albert Einstein
1  “It is our choices, Harry, that show what we t...     J.K. Rowling
2  “There are only two ways to live your life. On...  Albert Einstein
3  “The person, be it gentleman or lady, who has ...      Jane Austen
4  “Imperfection is beauty, madness is genius and...   Marilyn Monroe


#### - Usando Scrapy

**Scrapy** es un framework completo para web scraping a gran escala. Es más robusto que BeautifulSoup y está diseñado para proyectos de scraping complejos con múltiples páginas. Incluye manejo automático de peticiones, seguimiento de enlaces, pipelines de procesamiento de datos y respeto a reglas de robots.txt.

Se muestra un ejemplo en el archivo `scrapy_quotes.py`

#### - Usando Selenium

**Selenium** permite automatizar un navegador web real, útil para páginas que cargan contenido dinámicamente con JavaScript, sitios con múltiples páginas o contenido que aparece después de interacciones del usuario.

**ChromeDriver** es el programa que Selenium utiliza para controlar Chrome. Selenium 4.6+ incluye Selenium Manager que lo descarga y gestiona automáticamente según la versión de Chrome instalada.

En este ejemplo inicializamos un navegador Chrome automatizado, navegamos a través de múltiples páginas haciendo clic en "Siguiente" y recopilamos todas las citas.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd

# -----------------------------
# Configuración del navegador
# -----------------------------
options = Options()
# options.add_argument("--headless")
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)
driver.get("https://quotes.toscrape.com/")

wait = WebDriverWait(driver, 10)

# -----------------------------
# Scraping
# -----------------------------
data = []

while True:
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "quote")))

    quotes = driver.find_elements(By.CLASS_NAME, "quote")

    for quote in quotes:
        text = quote.find_element(By.CLASS_NAME, "text").text
        author = quote.find_element(By.CLASS_NAME, "author").text
        data.append({
            "quote": text,
            "author": author
        })

    try:
        next_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "li.next > a"))
        )
        next_button.click()
    except:
        break

driver.quit()

df = pd.DataFrame(data)
print(df.head())
print(df.shape)

#df.to_csv("selenium_quotes_all_pages.csv", index=False, encoding="utf-8")
#print("Datos guardados en selenium_quotes_all_pages.csv")


                                               quote           author
0  “The world as we have created it is a process ...  Albert Einstein
1  “It is our choices, Harry, that show what we t...     J.K. Rowling
2  “There are only two ways to live your life. On...  Albert Einstein
3  “The person, be it gentleman or lady, who has ...      Jane Austen
4  “Imperfection is beauty, madness is genius and...   Marilyn Monroe
(100, 2)


#### - Usando Playwright

**Playwright** es una herramienta moderna de automatización de navegadores desarrollada por Microsoft. Es similar a Selenium pero más rápido, con mejor API y soporte nativo para operaciones asíncronas.

**Ventajas sobre Selenium**:
- Más rápido y eficiente en recursos
- API más moderna y consistente
- Soporte nativo para múltiples navegadores (Chromium, Firefox, WebKit)
- Mejor manejo de esperas automáticas
- Capturas de pantalla y grabación de videos integrados

**Instalación**: Después de instalar playwright con pip, debes instalar los navegadores ejecutando: `playwright install`

Se muestra un ejemplo en el archivo `playwright_quotes.py`

### 3) APIs

Las **APIs** (Application Programming Interfaces) permiten acceder a datos de forma estructurada mediante solicitudes HTTP.

Una **API Key** es un código único que identifica y autentica a un usuario o aplicación. Sirve para controlar el acceso y rastrear el uso. Nunca compartas tu API key públicamente y ten en cuenta que muchas APIs gratuitas tienen límites de solicitudes por día/hora.

Para usar la API de OpenWeatherMap necesitas crear una cuenta gratuita en https://openweathermap.org/, generar tu API Key en la sección "API keys" de tu perfil y reemplazar el valor de `api_key` en el código con tu propia clave.

#### Ejemplo de la API de openweathermap

In [ ]:
import requests

# Tu clave de API (reemplaza con tu propia clave)
api_key = "60ccaadc128d87ca16cfffa9b2079ca6"

# Lista de ciudades para las cuales se quiere obtener el clima
cities = ["Lima", "Buenos Aires", "Madrid", "New York", "Tokyo"]

# Lista para almacenar los datos de cada ciudad
weather_data_list = []

# Hacer consultas para cada ciudad
for city in cities:
    # URL de la API de OpenWeatherMap para obtener el clima actual
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"

    # Realizamos la solicitud GET a la API
    response = requests.get(url)

    # Verificamos si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertimos la respuesta JSON en un diccionario de Python
        data = response.json()

        # Extraemos la información relevante del clima
        weather_data = {
            "City": data["name"],
            "Country": data["sys"]["country"],
            "Temperature (°C)": data["main"]["temp"],
            "Humidity (%)": data["main"]["humidity"],
            "Weather Description": data["weather"][0]["description"],
            "Pressure (hPa)": data["main"]["pressure"],
            "Wind Speed (m/s)": data["wind"]["speed"],
            "Wind Direction (°)": data["wind"]["deg"],  # Dirección del viento
            "Visibility (m)": data.get("visibility", "N/A"),  # Visibilidad
            "Latitude": data["coord"]["lat"],  # Latitud
            "Longitude": data["coord"]["lon"],  # Longitud
            "Date": pd.Timestamp.now()
        }

        # Agregamos el diccionario de esta ciudad a la lista
        weather_data_list.append(weather_data)

    else:
        print(f"Error al obtener datos para {city}: {response.status_code}")

# Convertimos la lista de diccionarios en un DataFrame
df = pd.DataFrame(weather_data_list)
df

,City,Country,Temperature (°C),Humidity (%),Weather Description,Pressure (hPa),Wind Speed (m/s),Wind Direction (°),Visibility (m),Latitude,Longitude,Date
0,Lima,PE,24.94,75,clear sky,1011,4.12,170,10000,-12.0432,-77.0282,2026-01-13 19:20:48.390293
1,Buenos Aires,AR,25.73,77,clear sky,1011,6.17,100,10000,-34.6132,-58.3772,2026-01-13 19:20:48.667271
2,Madrid,ES,7.87,90,overcast clouds,1016,2.57,160,10000,40.4165,-3.7026,2026-01-13 19:20:49.004095
3,New York,US,6.64,55,clear sky,1011,6.69,180,10000,40.7143,-74.0060,2026-01-13 19:20:49.203201
4,Tokyo,JP,7.42,42,few clouds,1012,4.63,300,10000,35.6895,139.6917,2026-01-13 19:20:49.412843


En este ejemplo:
1. Definimos una lista de ciudades a consultar
2. Para cada ciudad hacemos una solicitud GET a la API con nuestra clave
3. Extraemos información relevante del JSON de respuesta (temperatura, humedad, viento, etc.)
4. Consolidamos todos los datos en un DataFrame para análisis comparativo